In [ ]:
# %%
import os
import shutil

# %%
# more current chroma database
if os.path.exists('./chroma'):
    shutil.rmtree('./chroma')

# %%
# with open('secret/openai_api_key.txt') as f:
#     openai_api_key = f.read()

# %%
# from langchain_openai import OpenAIEmbeddings
# embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)

from langchain_ollama import OllamaEmbeddings
embedding_llm = OllamaEmbeddings(model="llama3.2")

# %%
from langchain_community.document_loaders import TextLoader

# %% [markdown]
# # Files & Loading

# %%
from pathlib import Path

# %%
docs_base = Path("./docs")
files_all = docs_base.rglob("*")
excludes = [".DS_Store", ".ipynb_checkpoints"]
files = []
for f in files_all:
    to_exclude = False
    for ex in excludes:
        to_exclude = to_exclude or f.match(ex)
    if not to_exclude and f.is_file():
        files.append(f)

# %%
files

# %%
documents = []
for file in files:
    print(file)
    loader = TextLoader(str(file.absolute()),encoding='utf-8')
    documents += loader.load()

# %%
documents

# %%
documents[0].__dict__

# %% [markdown]
# # Indexing with Chroma

# %%
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

# %%
db = Chroma.from_documents(documents, embedding_llm, persist_directory="./chroma",
                          collection_name="planetbucks")

# %%
db.persist()

# %%





In [ ]:
# %%
import os
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

# %% [markdown]
# # Create Embedding LLM (OpenAI)

# %%
# with open('./openai_api_key.txt') as f:
#     openai_api_key = f.read()
# embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)
embedding_llm = OllamaEmbeddings(model="llama3.2")

# %% [markdown]
# # Load Databases

# %%
db = Chroma(persist_directory='./chroma', embedding_function=embedding_llm,
            collection_name="planetbucks")

# %% [markdown]
# # Using Chroma vector database

# %% [markdown]
# ## Similarity search

# %%
query = "opening hours"
result = db.similarity_search_with_relevance_scores(query)
# result = db.similarity_search_with_score(query)
print(len(result))

# %%
result

# %%
result = db.similarity_search_with_relevance_scores(query, k=2)
print(len(result))

# %%
result

# %% [markdown]
# ## MMR (Maximum marginal relevance)
# - MMR = iteratively find documents that are dissimilar to previous results.
# - It could improve performance for retrievals.

# %%
db.max_marginal_relevance_search(query, k=2)

# %% [markdown]
# # Retriever

# %%
retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 2, 'fetch_k': 5})

# %%
retriever.invoke(query)




In [ ]:
# %%
import os
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

# %% [markdown]
# # Create Embedding LLM (OpenAI)

# %%
# with open('./openai_api_key.txt') as f:
#     openai_api_key = f.read()
# embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)
embedding_llm = OllamaEmbeddings(model="llama3.2")

# %% [markdown]
# # Load Databases

# %%
db = Chroma(persist_directory='./chroma', embedding_function=embedding_llm,
            collection_name="planetbucks")

# %% [markdown]
# # Creating RAG tools

# %%
retriever_tool = create_retriever_tool(
    db.as_retriever(search_type='mmr'),
    name = "planetbucks_search",
    description = """Search for information about PlanetBucks store, including store information, 
    coffee drink menus, specialty coffee beans menu, and bean fact sheet.""",
)

# %% [markdown]
# # (Legacy) Create AgentExecutor

# %%
# # from langchain_openai import ChatOpenAI
# # from langchain_ollama import OllamaLLM as Ollama
# from langchain_ollama.chat_models import ChatOllama
# from langchain.prompts import ChatPromptTemplate

# # llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
# llm = ChatOllama(model="llama3.2", temperature=0)

# %%
# system_message = """You are a helpful assistant working at a coffee shop.
# You can use a given chat history and given tools to respond to a user.
# Your character is a polite and friendly female.
# You answer concisely, without introduction or appending.
# If you do not know the answer, just say 'I do not know.'
# When you calculate the price of an ordered item, you should think step-by-step.
# """

# %%
# prompt = ChatPromptTemplate.from_messages(
#     [
#         SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_message)),
#         MessagesPlaceholder(
#             variable_name="chat_history", optional=True
#         ),  # Where the memory will be stored.
#         HumanMessagePromptTemplate.from_template(
#             "{input}"
#         ),  # Where the human input will injected
#         MessagesPlaceholder(
#             variable_name="agent_scratchpad"
#         ),  # Where the memory will be stored.
#     ]
# )

# %%
# tools = [retriever_tool, ]

# %%
# from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
# from langchain.memory import ConversationBufferMemory

# agent = create_openai_functions_agent(llm, tools, prompt,)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# %% [markdown]
# ## ChatInterface

# %%
# def wrapper_chat_history(chat_history, memory):
#     chat_history = []
#     for m in memory.chat_memory.messages:
#         chat_history.append(m.content)
#     return chat_history

# %%
# def converse(message, chat_history):
#     response = agent_executor.invoke({"input": message})
#     chat_history = wrapper_chat_history(chat_history, memory)
#     return response['output']

# %%
# import gradio as gr

# demo = gr.ChatInterface(fn=converse)

# demo.launch(share=False)


# %% [markdown]
# # Agent with LangGraph

# %%
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver

# %%
llm = ChatOllama(model="llama3.2", temperature=0)
tools = [retriever_tool, ]

SYSTEM_MESSAGE = """You are a helpful receptionist at PlanetBucks, which is a coffee cafe. Your name i Echo.
You wil answer politely but playfully since it's Christmas festival time now.
"""

memory = MemorySaver()

langgraph_agent_executor = create_react_agent(
    llm,
    tools,
    state_modifier=SYSTEM_MESSAGE,
    checkpointer=memory
)

# %%
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        langgraph_agent_executor.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

# %%
config = {"configurable": {"thread_id": "thread1"}}

query = "I'm Ismail. Hello."

response = langgraph_agent_executor.invoke(
    {"messages": [("human", query)]},
    config
)

# %%
response

# %%
query = "What is opening hour?"

response = langgraph_agent_executor.invoke({"messages": [("human", query)]}, config)

# %%
response

# %%
len(response['messages'])

# %%
response['messages']

# %%
query = "What's your name?"

response = langgraph_agent_executor.invoke({"messages": [("human", query)]}, config)

# %%
response

# %% [markdown]
# ## ChatInterface

# %%
import gradio as gr

# %%
config = {"configurable": {"thread_id": "thread1"}}

llm = ChatOllama(model="llama3.2", temperature=0)
tools = [retriever_tool, ]


SYSTEM_MESSAGE = """You are a helpful receptionist at PlanetBucks, which is a coffee cafe. Your name i Echo.
You wil answer politely but playfully since it's Christmas festival time now.
"""

# %%
try:
    demo.close()
except:
    pass

global chat_history
chat_history = []

memory = MemorySaver()

langgraph_agent_executor = create_react_agent(
    llm,
    tools,
    state_modifier=SYSTEM_MESSAGE,
    checkpointer=memory
)

def chat(message, history):
    global chat_history
    response = langgraph_agent_executor.invoke(
        {"messages": [("human", message)]},
        config
        )
    chat_history = response
    return response['messages'][-1].content

demo = gr.ChatInterface(fn=chat)

demo.launch(share=False)




# %%
chat_history

# %%



